# Errors & Exception Handling in Python

## Definitions & Concepts

- Exception: An event detected during program execution that interrupts normal flow.
- Error: Broad category often used interchangeably with exception (e.g., SyntaxError, RuntimeError).
- try/except: Block to catch and handle exceptions.
- else: Runs if no exception occurred in the try block.
- finally: Runs always (cleanup code).
- raise: Explicitly raise an exception.
- Custom exceptions: Subclass Exception for domain-specific errors.
- Exception chaining: Use `raise ... from ...` to preserve context.
- Assertions: `assert` to check invariants (used mainly for debugging / testing).
- Best practices: catch specific exceptions, avoid bare except, cleanup in finally / context managers, log exceptions.

### Example 1 — Basic try / except (handle ZeroDivisionError)

In [1]:
try:
    x = 1 / 0
except ZeroDivisionError:
    print('Caught division by zero')
else:
    print('No error')
finally:
    print('Cleanup actions run always')

Caught division by zero
Cleanup actions run always


### Example 2 — Catch multiple specific exception types

In [2]:
def parse_int(s: str):
    try:
        return int(s)
    except (ValueError, TypeError) as exc:
        print('Could not parse:', exc)

parse_int('10')
parse_int('ten')
parse_int(None)

Could not parse: invalid literal for int() with base 10: 'ten'
Could not parse: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'


### Example 3 — Use else and finally for flow control and cleanup

In [3]:
f = None
try:
    f = open('nonexistent_file.txt', 'r')
    data = f.read()
except FileNotFoundError:
    print('File not found — handled')
else:
    print('Read succeeded')
finally:
    # safe cleanup
    if f is not None:
        f.close()
        print('File closed')
    else:
        print('No file to close')

File not found — handled
No file to close


### Example 4 — Raise exceptions (including custom exception)

In [4]:
class ValidationError(ValueError):
    """Custom exception for validation errors."""
    pass


def validate_age(age):
    if age < 0:
        raise ValidationError(f'age must be non-negative, got {age}')
    return True

try:
    validate_age(-5)
except ValidationError as e:
    print('Validation failed:', e)

Validation failed: age must be non-negative, got -5


### Example 5 — Re-raising and logging the original exception

In [5]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('example5')

try:
    {}['missing']  # force KeyError
except KeyError as e:
    logger.exception('Key not found, re-raising as RuntimeError')
    raise RuntimeError('higher-level error') from e

ERROR:example5:Key not found, re-raising as RuntimeError
Traceback (most recent call last):
  File "C:\Users\Vivobook\AppData\Local\Temp\ipykernel_22488\3716296333.py", line 6, in <module>
    {}['missing']  # force KeyError
    ~~^^^^^^^^^^^
KeyError: 'missing'


RuntimeError: higher-level error

### Example 6 — Assertion for internal checks (not for user input validation)

Use assert to express invariants during development or tests.

In [6]:
def average(numbers):
    assert len(numbers) > 0, 'numbers must not be empty'
    return sum(numbers) / len(numbers)

try:
    average([])
except AssertionError as e:
    print('AssertionError:', e)

print('Average of [1,2,3] =', average([1,2,3]))

AssertionError: numbers must not be empty
Average of [1,2,3] = 2.0


### Example 7 — Context manager __enter__/__exit__ handles exceptions (resource management)

In [7]:
class DummyResource:
    def __enter__(self):
        print('acquired')
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('released')
        # Returning False propagates exception; True would suppress it
        return False

try:
    with DummyResource():
        print('inside block')
        raise ValueError('demo')
except ValueError:
    print('ValueError propagated and handled outside')

acquired
inside block
released
ValueError propagated and handled outside


### Example 8 — Exception chaining with `raise ... from ...`

In [8]:
def read_config(text):
    try:
        return int(text)
    except Exception as e:
        # chain lower-level parsing error into a domain-specific error
        raise RuntimeError('config parse failed') from e

try:
    read_config('not-int')
except RuntimeError as e:
    print('Chained exception example:')
    import traceback
    traceback.print_exc()

Chained exception example:


Traceback (most recent call last):
  File "C:\Users\Vivobook\AppData\Local\Temp\ipykernel_22488\1360631334.py", line 3, in read_config
    return int(text)
ValueError: invalid literal for int() with base 10: 'not-int'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Vivobook\AppData\Local\Temp\ipykernel_22488\1360631334.py", line 9, in <module>
    read_config('not-int')
    ~~~~~~~~~~~^^^^^^^^^^^
  File "C:\Users\Vivobook\AppData\Local\Temp\ipykernel_22488\1360631334.py", line 6, in read_config
    raise RuntimeError('config parse failed') from e
RuntimeError: config parse failed


### Example 9 — Wrap exceptions when crossing module boundaries (convert to stable public exception)

In [9]:
class PublicAPIError(Exception):
    """Stable exception type exposed by a library's API."""
    pass

# internal function may raise many types; expose a single API-level error

def library_action(value):
    try:
        # internal operations that can raise different errors
        return 1 / int(value)
    except Exception as e:
        # hide internal details, raise API-specific error
        raise PublicAPIError('library_action failed') from e

try:
    library_action('0')
except PublicAPIError as e:
    print('Handled PublicAPIError:', e)
    import traceback
    print('Underlying cause:')
    traceback.print_tb(e.__traceback__)

Handled PublicAPIError: library_action failed
Underlying cause:


  File "C:\Users\Vivobook\AppData\Local\Temp\ipykernel_22488\2624578191.py", line 16, in <module>
    library_action('0')
    ~~~~~~~~~~~~~~^^^^^
  File "C:\Users\Vivobook\AppData\Local\Temp\ipykernel_22488\2624578191.py", line 13, in library_action
    raise PublicAPIError('library_action failed') from e


### Example 10 — Best-practice patterns and safe cleanup

- Catch specific exceptions (avoid bare `except:`).
- Use `finally` or context managers for cleanup.
- Log exceptions where appropriate.
- Prefer re-raising with context when converting exceptions.

Below is a compact, practical pattern combining validation, try/except/else/finally, and logging.

In [10]:
import logging
logger = logging.getLogger('example10')

def process(filename):
    if not isinstance(filename, str):
        raise TypeError('filename must be a string')

    f = None
    try:
        f = open(filename, 'r')
        data = f.read()
        # process data (pretend)
        result = len(data)
    except FileNotFoundError:
        logger.error('file not found: %s', filename)
        raise
    except Exception as e:
        logger.exception('unexpected error while processing %s', filename)
        raise
    else:
        print('processed OK, length =', result)
    finally:
        if f is not None:
            f.close()
            print('file closed in finally')

# Run with a missing file to show logging + cleanup
try:
    process('this_file_does_not_exist.txt')
except Exception:
    print('Top-level handler received an exception (as expected)')

ERROR:example10:file not found: this_file_does_not_exist.txt


Top-level handler received an exception (as expected)


## Summary

Catch specific exceptions, use else/finally, prefer context managers for resources, convert exceptions across module boundaries responsibly, and use logging for diagnostics. Use custom exceptions to make error handling clearer for callers.